In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt


In [123]:

cotas = pd.read_excel(f"{os.getcwd()}\\Cotas.xlsx",).fillna(0)
def interpolSideways(tabelaCotas):
    
    #storeS é reponsável por ir armazenando os valores de newLine que são
    #valores novos de cada linha além dos já existentes
    storeS=[]
    
    #Formando a primera linha da nova matriz
    newLine=[tabelaCotas[0,0],tabelaCotas[0,1]]
    for j in range(2,len(tabelaCotas[0])):
        halfVal=tabelaCotas[0,j]-(tabelaCotas[0,j]-tabelaCotas[0,j-1])/2
        newLine.append(halfVal)
        newLine.append(tabelaCotas[0,j])
    
    storeS.append(newLine)    
       
    #Formando todas as novas linhas da nova matriz. newLine é resetado toda
    #vez que este loop é executado.
    for i in range(1,len(tabelaCotas)):
        
        #Calculo dos h's
        storeH=[]
        beginCounter=len(tabelaCotas[0])
        for j in range(2,len(tabelaCotas[0])):
            h=tabelaCotas[0,j]-tabelaCotas[0,j-1]
            if tabelaCotas[i,j]-tabelaCotas[i,j-1]!=0:
                storeH.append(h)
                beginCounter-=1
        
        #Calculo da matriz A e da matrz b
        A=np.zeros((len(storeH)-1,len(storeH)+1))
        b=np.zeros((len(storeH)-1))
        for m in range(len(storeH)-1):
            A[m,m]=storeH[m]
            A[m,m+1]=2*(storeH[m]+storeH[m+1])
            A[m,m+2]=storeH[m+1]
            
            b[m]=(tabelaCotas[i,m+beginCounter+1]-tabelaCotas[i,m+beginCounter])/storeH[m+1]-(tabelaCotas[i,m+beginCounter]-tabelaCotas[i,m+beginCounter-1])/storeH[m]
        
        A=A[:,1:len(A[0])-1] #Removendo primeira e última coluna para a operação
        b=6*b
        
        #Calculo da matriz x com os coeficientes g
        matrixX=np.linalg.solve(A,b)
        matrixX=np.append(matrixX, 0.0) #Adicionando o primeiro e último zero que foram retirados
        matrixX=np.insert(matrixX, 0, 0.0, axis=0)
        
        #Calculo das splines entre cada ponto e seu valor
        newLine=[tabelaCotas[i,0],tabelaCotas[i,1]]
        zeroCounter=2
        for k in range(2,len(tabelaCotas[0])):
            
            if tabelaCotas[i,k]-tabelaCotas[i,k-1]==0:
                newLine.append(0.0)
                newLine.append(tabelaCotas[i,k])
                zeroCounter+=1
            else:
                #Calculo dos coeficientes
                a=(matrixX[k-zeroCounter+1]-matrixX[k-zeroCounter])/(6*storeH[k-zeroCounter])
                b=matrixX[k-zeroCounter+1]/2
                c=(tabelaCotas[i,k]-tabelaCotas[i,k-1])/storeH[k-zeroCounter]+(2*storeH[k-zeroCounter]*matrixX[k-zeroCounter+1]+matrixX[k-zeroCounter]*storeH[k-zeroCounter])/6
                d=tabelaCotas[i,k]
                
                #Calculo do ponto entre dois conhecidos e append para ir construindo a nova linha
                halfVal=tabelaCotas[0,k]-(tabelaCotas[0,k]-tabelaCotas[0,k-1])/2
                s=a*(halfVal-tabelaCotas[0,k])**3+b*(halfVal-tabelaCotas[0,k])**2+c*(halfVal-tabelaCotas[0,k])+d

                newLine.append(s)
                newLine.append(tabelaCotas[i,k])
        
        storeS.append(newLine)
        
    #Transformando a lista final em matriz e atribuindo a tabela de cotas
    tabelaCotas=np.asarray(storeS)
    return tabelaCotas

def interpolDownwards(tabelaCotas):
    storeS=[]
    
    #Formando a primera coluna da nova matriz
    newColumn=[tabelaCotas[0,0],tabelaCotas[1,0]]
    for i in range(2,len(tabelaCotas)):
        halfVal=tabelaCotas[i,0]-(tabelaCotas[i,0]-tabelaCotas[i-1,0])/2
        newColumn.append(halfVal)
        newColumn.append(tabelaCotas[i,0])
    
    storeS.append(newColumn)    
       
    #Formando todas as novas colunas da nova matriz. newColumn é resetado toda
    #vez que este loop é executado.
    for j in range(1,len(tabelaCotas[0])):
        
        #Calculo dos h's
        storeH=[]
        beginCounter=len(tabelaCotas)
        for i in range(2,len(tabelaCotas)):
            h=tabelaCotas[i,0]-tabelaCotas[i-1,0]
            
            #Adicionamos apenas os h's diferentes de zero para nossa matriz
            #storeH para que calculemos apenas com no máximo 1 ponto igual a zero
            if tabelaCotas[i,j]-tabelaCotas[i-1,j]!=0:
                storeH.append(h)           
                beginCounter-=1
            
            #Para o caso de cada coluna precisamos fazer esta condição para não
            #obtermos erro no beginCounter
            elif tabelaCotas[i,j]-tabelaCotas[i-1,j]==0 and i>len(tabelaCotas)/2:
                beginCounter-=1
            
        #Calculo da matriz A e da matrz b
        A=np.zeros((len(storeH)-1,len(storeH)+1))
        b=np.zeros((len(storeH)-1))
        for m in range(len(storeH)-1):
            A[m,m]=storeH[m]
            A[m,m+1]=2*(storeH[m]+storeH[m+1])
            A[m,m+2]=storeH[m+1]
            
            b[m]=(tabelaCotas[m+beginCounter+1,j]-tabelaCotas[m+beginCounter,j])/storeH[m+1]-(tabelaCotas[m+beginCounter,j]-tabelaCotas[m+beginCounter-1,j])/storeH[m]
        A=A[:,1:len(A[0])-1] #Removendo primeira e última coluna para a operação
        b=6*b
        
        #Calculo da matriz x com os coeficientes g
        matrixX=np.linalg.solve(A,b)
        matrixX=np.append(matrixX, 0.0) #Adicionando o primeiro e último zero que foram retirados
        matrixX=np.insert(matrixX, 0, 0.0, axis=0)
     
        #Calculo das splines entre cada ponto e seu valor
        newColumn=[tabelaCotas[0,j],tabelaCotas[1,j]]
        zeroCounter=2
        for k in range(2,len(tabelaCotas)):
            
            if tabelaCotas[k,j]-tabelaCotas[k-1,j]==0:
                newColumn.append(0.0)
                newColumn.append(tabelaCotas[k,j])
                zeroCounter+=1
            else:
                #Calculo dos coeficientes
                a=(matrixX[k-zeroCounter+1]-matrixX[k-zeroCounter])/(6*storeH[k-zeroCounter])
                b=matrixX[k-zeroCounter+1]/2
                c=(tabelaCotas[k,j]-tabelaCotas[k-1,j])/storeH[k-zeroCounter]+(2*storeH[k-zeroCounter]*matrixX[k-zeroCounter+1]+matrixX[k-zeroCounter]*storeH[k-zeroCounter])/6
                d=tabelaCotas[k,j]
            
                #Calculo do ponto entre dois pontos conhecidos e append para ir 
                #construindo a nova coluna
                halfVal=tabelaCotas[k,0]-(tabelaCotas[k,0]-tabelaCotas[k-1,0])/2
                s=a*(halfVal-tabelaCotas[k,0])**3+b*(halfVal-tabelaCotas[k,0])**2+c*(halfVal-tabelaCotas[k,0])+d
                newColumn.append(s)
                newColumn.append(tabelaCotas[k,j])
        
        storeS.append(newColumn)
    
    #Transformando a lista em matriz e a tranpondo para obter ela em forma de colunas
    tabelaCotas=np.asarray(storeS).transpose()
    return tabelaCotas

def nInterpolBoth(tabelaCotas, n):
    for i in range(1,n+1):
        tabelaCotas=interpolDownwards(interpolSideways(tabelaCotas))

    return tabelaCotas

def nInterpolDownwards(tabelaCotas, n):
    for i in range(1,n+1):
        tabelaCotas=interpolDownwards(tabelaCotas)

    return tabelaCotas

def nInterpolSideways(tabelaCotas, n):
    for i in range(1,n+1):
        tabelaCotas=interpolSideways(tabelaCotas)

    return tabelaCotas

nInterpolBoth(tabelaCotas, 1)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000,0.000000,0.711000,1.422000,2.844000,4.266000,5.688000,7.110000,8.532000,9.954000,11.376000,12.798000
1,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.651998,3.779004,5.301003,6.401998,7.133003
2,6.275,0.281195,0.753759,0.838000,0.838000,0.995999,1.957999,4.227998,6.270001,7.667001,8.452998,8.776005
3,12.550,0.542604,1.547002,1.869998,2.201001,2.791001,4.240005,6.302001,8.045999,9.169997,9.808002,10.133996
4,18.825,1.279997,2.502005,3.164001,3.941001,4.934001,6.389003,7.982000,9.274004,10.165996,10.663995,10.923003
5,25.100,2.200002,3.589005,4.630999,5.966998,7.006005,8.185005,9.281002,10.168995,10.849996,11.203003,11.374997
6,31.375,2.929996,5.019004,6.314996,7.826000,8.762998,9.610994,10.287997,10.814997,11.208001,11.409995,11.490000
7,37.650,4.299995,6.520001,7.998005,9.358996,10.061000,10.534997,10.899000,11.195006,11.361002,11.490000,11.490000
8,43.925,5.759994,8.137000,9.465002,10.570995,10.945994,11.158996,11.380994,11.490000,11.490000,11.490000,11.490000
9,50.200,7.132303,9.426005,10.514005,11.216998,11.348995,11.490000,11.490000,11.490000,11.490000,11.490000,11.490000


,0,2,3,4,5,6,7,8,9,10,11
0,0.000,0.711000,1.422000,2.844000,4.266000,5.688000,7.110000,8.532000,9.954000,11.376000,12.798000
1,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,1.651998,3.779004,5.301003,6.401998,7.133003
2,6.275,0.753759,0.838000,0.838000,0.995999,1.957999,4.227998,6.270001,7.667001,8.452998,8.776005
3,12.550,1.547002,1.869998,2.201001,2.791001,4.240005,6.302001,8.045999,9.169997,9.808002,10.133996
4,18.825,2.502005,3.164001,3.941001,4.934001,6.389003,7.982000,9.274004,10.165996,10.663995,10.923003
5,25.100,3.589005,4.630999,5.966998,7.006005,8.185005,9.281002,10.168995,10.849996,11.203003,11.374997
6,31.375,5.019004,6.314996,7.826000,8.762998,9.610994,10.287997,10.814997,11.208001,11.409995,11.490000
7,37.650,6.520001,7.998005,9.358996,10.061000,10.534997,10.899000,11.195006,11.361002,11.490000,11.490000
8,43.925,8.137000,9.465002,10.570995,10.945994,11.158996,11.380994,11.490000,11.490000,11.490000,11.490000
9,50.200,9.426005,10.514005,11.216998,11.348995,11.490000,11.490000,11.490000,11.490000,11.490000,11.490000


Unnamed: 0
A.P.        0.000000
ST 1        0.024473
ST 2        0.047224
ST 3        0.111401
ST 4        0.191471
ST 5        0.255004
ST 6        0.374238
ST 7        0.501305
ST 8        0.620740
ST 9        0.693464
AMIDSHIP    0.672472
ST 11       0.591819
ST 12       0.437772
ST 13       0.308964
ST 14       0.195822
ST 15       0.104439
ST 16       0.055614
ST 17       0.038033
ST 18       0.010009
ST 19       0.000000
F.P.        0.000000
Name: WL 0, dtype: float64 Unnamed: 0
A.P.        0.000000
ST 1        0.024473
ST 2        0.047224
ST 3        0.111401
ST 4        0.191471
ST 5        0.255004
ST 6        0.374238
ST 7        0.501305
ST 8        0.620740
ST 9        0.693464
AMIDSHIP    0.672472
ST 11       0.591819
ST 12       0.437772
ST 13       0.308964
ST 14       0.195822
ST 15       0.104439
ST 16       0.055614
ST 17       0.038033
ST 18       0.010009
ST 19       0.000000
F.P.        0.000000
Name: WL 0, dtype: float64


ValueError: Length of values (420) does not match length of index (4)